In [20]:
from blockchain import blockexplorer

latest = blockexplorer.get_block(blockexplorer.get_latest_block().hash)
latest?

SyntaxError: invalid syntax (<ipython-input-20-02571bc02842>, line 1)

In [18]:
blocks = [latest]
prev = []
prevHash = None
for i in range(0,50):
    prevHash=blocks[-1].previous_block
    b = blockexplorer.get_block(prevHash)
    blocks.append(b)
    prev = b.previous_block
    print(b.fee)

51244828
122838655
121558958
83782661


KeyboardInterrupt: 

In [ ]:
#import csv

#with open('transactions.csv', mode='w') as employee_file:
    #employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #employee_writer.writerow(['John Smith', 'Accounting', 'November'])

In [3]:
txFees = []
tx1 = block.transactions[2]
tx1.fee

80812

In [4]:
addr = blockexplorer.get_address("1Bf9sZvBHPFGVPX71WX2njhd1NXKv5y7v5")

In [5]:
addr.final_balance
t1 = addr.transactions[0]

In [7]:
t1.size

343